In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Testicular_Cancer"
cohort = "GSE62523"

# Input paths
in_trait_dir = "../../input/GEO/Testicular_Cancer"
in_cohort_dir = "../../input/GEO/Testicular_Cancer/GSE62523"

# Output paths
out_data_file = "../../output/preprocess/Testicular_Cancer/GSE62523.csv"
out_gene_data_file = "../../output/preprocess/Testicular_Cancer/gene_data/GSE62523.csv"
out_clinical_data_file = "../../output/preprocess/Testicular_Cancer/clinical_data/GSE62523.csv"
json_path = "../../output/preprocess/Testicular_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE62523_family.soft.gz', 'GSE62523_series_matrix.txt.gz']
Identified SOFT files: ['GSE62523_family.soft.gz']
Identified matrix files: ['GSE62523_series_matrix.txt.gz']

Background Information:
!Series_title	"Gene expression profiles of HMEC-1 after exposure to the chemotherapeutic drugs bleomycin and cisplatin with untreated samples as control"
!Series_summary	"Chemotherapy-related endothelial damage contributes to the early development of cardiovascular morbidity in testicular cancer patients. We aimed to identify relevant mechanisms of and search for candidate biomarkers for this endothelial damage."
!Series_summary	"Human micro-vascular endothelial cells (HMEC-1) were exposed to bleomycin or cisplatin with untreated samples as control. 18k cDNA microarrays were used. Gene expression differences were analysed at single gene level and in gene sets clustered in biological pathways and validated by qRT-PCR. Protein levels of a candidate biomarker were m

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Check gene expression data availability
# Based on the background information, this seems to be a gene expression dataset of HMEC-1 cell line
# The description mentions 18k cDNA microarrays were used
is_gene_available = True

# Step 2: Analyze variable availability

# 2.1 Data Availability
# The sample characteristics show this dataset is about HMEC-1 cell line exposed to chemotherapeutic drugs
# Looking at the characteristics, there's no specific information about testicular cancer patients, age, or gender
# This appears to be a cell line study, not a patient cohort with trait, age, or gender information

# For Testicular Cancer trait: Not directly available in cell line data
trait_row = None

# For Age: Not applicable for cell line data
age_row = None  

# For Gender: Not applicable for cell line data
gender_row = None

# 2.2 Data Type Conversion Functions
# Even though we won't use these functions since we don't have the data,
# we'll define them as required by the task structure

def convert_trait(value):
    """Convert trait value to binary format.
    Since this is a cell line study, this function won't be used."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    return None  # Not applicable for this dataset

def convert_age(value):
    """Convert age value to continuous format.
    Since this is a cell line study, this function won't be used."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    return None  # Not applicable for this dataset

def convert_gender(value):
    """Convert gender value to binary format.
    Since this is a cell line study, this function won't be used."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    return None  # Not applicable for this dataset

# Step 3: Save Metadata
# The dataset doesn't contain human trait data as it's a cell line study
is_trait_available = trait_row is not None

# Perform initial validation and save the cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# We skip this step since trait_row is None (clinical data not available for this type of study)


A new JSON file was created at: ../../output/preprocess/Testicular_Cancer/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1.1.1.1', '1.1.1.10', '1.1.1.11', '1.1.1.12', '1.1.1.13', '1.1.1.14',
       '1.1.1.15', '1.1.1.16', '1.1.1.17', '1.1.1.18', '1.1.1.19', '1.1.1.2',
       '1.1.1.20', '1.1.1.21', '1.1.1.22', '1.1.1.23', '1.1.1.3', '1.1.1.4',
       '1.1.1.5', '1.1.1.6'],
      dtype='object', name='ID')

Gene expression data shape: (25392, 104)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be probe identifiers and not standard human gene symbols
# They follow a pattern like "1.1.1.1", "1.1.1.2", etc., which is not consistent with standard gene symbols
# Standard human gene symbols are typically alphabetic (e.g., TP53, BRCA1, MYC)
# These identifiers will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1.1.1.1', '1.1.1.2', '1.1.1.3', '1.1.1.4', '1.1.1.5'], 'Meta Row': ['1', '1', '1', '1', '1'], 'Meta Column': [1.0, 1.0, 1.0, 1.0, 1.0], 'Row': [1.0, 1.0, 1.0, 1.0, 1.0], 'Column': [1.0, 2.0, 3.0, 4.0, 5.0], 'Gene ID': ['c_Cy-3 landmark', 'c_Cy-3 landmark', 'c_Cy-5 landmark', 'AK056492', 'AK057091'], 'GB_ACC': [nan, nan, nan, 'AK056492', 'AK057091'], 'Gene symbol': [nan, nan, nan, 'ZNF827', 'FAM120B'], 'SPOT_ID': ['c_Cy-3 landmark', 'c_Cy-3 landmark', 'c_Cy-5 landmark', nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for mapping
# From the gene annotation preview, we need to use:
# - 'ID' column as the gene identifier that matches the index in gene_data
# - 'Gene symbol' column for the human gene symbols

# 2. Create a mapping dataframe with the relevant columns
prob_col = 'ID'
gene_col = 'Gene symbol'
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Look at the mapping data 
print("Gene mapping preview:")
print(mapping_data.head())
print(f"Mapping data shape: {mapping_data.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Print information about the resulting gene expression data
print("\nAfter mapping to gene symbols:")
print(f"Gene expression data shape: {gene_data.shape}")
print("First 10 genes:")
print(gene_data.index[:10])


Gene mapping preview:
        ID     Gene
3  1.1.1.4   ZNF827
4  1.1.1.5  FAM120B
5  1.1.1.6   ZNF655
6  1.1.1.7     KRAS
7  1.1.1.8    RPP14
Mapping data shape: (18201, 2)



After mapping to gene symbols:
Gene expression data shape: (15399, 104)
First 10 genes:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AADAC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data using the provided function
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since previous steps determined that trait_row is None and is_trait_available is False,
# we should use is_final=False for the validation (which doesn't require is_biased)
# Conduct quality check with correct trait availability (False)
is_usable = validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=False  # Correctly report that trait data is not available
)

# Create a minimal clinical dataframe for linking (note: this is empty in this case)
clinical_df = pd.DataFrame()

# Save empty clinical data for consistency
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data (empty) saved to {out_clinical_data_file}")

print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
print("Dataset is not usable for trait association study as it doesn't contain trait data.")

Normalized gene data shape: (15073, 104)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AADAT']


Normalized gene data saved to ../../output/preprocess/Testicular_Cancer/gene_data/GSE62523.csv
Clinical data (empty) saved to ../../output/preprocess/Testicular_Cancer/clinical_data/GSE62523.csv
Data quality check result: Not usable
Dataset is not usable for trait association study as it doesn't contain trait data.
